# Trying Sherpa

I was already making code ready to post a question on StackOverflow, when I found this question and the related answers:

https://stackoverflow.com/questions/15345906/user-specified-function-in-mpfit

The first answer links to the modelling and fitting package `sherpa` which I have never heard of before, but it seems to be super well documented and writen with an object-oriented approach, so I decided to see whether that would actually be useful for us. Here's the link to it:

http://cxc.harvard.edu/contrib/sherpa/

`pip install sherpa` did it.

I started by just copy-pasting the first example from their documentation:

https://sherpa.readthedocs.io/en/4.11.0/quick.html

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt

os.chdir('../')
from config import CONFIG_INI
import margmodule as marg

## Write your own model class

First, all the sherpa imports.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sherpa.models import model
from sherpa.data import Data1D
from sherpa.plot import DataPlot
from sherpa.plot import ModelPlot
from sherpa.fit import Fit
from sherpa.stats import LeastSq
from sherpa.optmethods import LevMar
from sherpa.stats import Chi2
from sherpa.plot import FitPlot

### Define the model mathod

This calculates a model form given parameters *pars* and an independent variable *x*.

**This was moved to `margmodule.py`**

### Define the user model class

All parameters have at least a name and an initial guess. Make sure you don't forget a parameter in the initialization of the super class.

**This was moved to `margmodule.py`**

## Import the data

In [ ]:
# Import data
localDir = CONFIG_INI.get('data_paths', 'local_path')
curr_model = CONFIG_INI.get('data_paths', 'current_model')
dataDir = os.path.join(localDir, os.path.join(localDir, CONFIG_INI.get('data_paths', 'data_path')), curr_model)
x, y, err, sh = np.loadtxt(os.path.join(dataDir, 'W17_white_lightcurve_test_data.txt'),
                           skiprows=7, unpack=True)

In [ ]:
# Plot the data with matplotlib
plt.scatter(x, y, s=5)
plt.title("Data")

## Create Sherpa data object from your data

In [ ]:
# Create Sherpa data object
data = Data1D('Data', x, y, staterror=err)
print(data)

In [ ]:
# Plot the data with Sherpa
dplot = DataPlot()    # create plot object
dplot.prepare(data)      # prepare data for plotting
dplot.plot()          # plot

## Instantiate the model

In [ ]:
# Define the model
tmodel = marg.Transit()
print(tmodel)

In [ ]:
# Freeze some parameters
tmodel.inclin.frozen = True
print(tmodel.inclin.frozen)

tmodel.msmpr.frozen = True
tmodel.ecc.frozen = True
tmodel.omega.frozen = True
tmodel.period.frozen = True
tmodel.tzero.frozen = True
tmodel.c1.frozen = True
tmodel.c2.frozen = True
tmodel.c3.frozen = True
tmodel.c4.frozen = True
tmodel.m_fac.frozen = True
tmodel.hstp1.frozen = True
tmodel.hstp2.frozen = True
tmodel.hstp3.frozen = True
tmodel.hstp4.frozen = True
tmodel.xshift1.frozen = True
tmodel.xshift2.frozen = True
tmodel.xshift3.frozen = True
tmodel.xshift4.frozen = True

print(tmodel)

In [ ]:
# Plot the model
mplot = ModelPlot()
mplot.prepare(data, tmodel)
mplot.plot()

## The fitting

### Select the statistics and optimizer

In [ ]:
# Select statistics and optimizer
stat = Chi2()
opt = LevMar()

### Instantiate the fit object and fit

In [ ]:
# Fit
tfit = Fit(data, tmodel, stat=stat, method=opt)    # Instantiate fit object
tres = tfit.fit()     # do the fit
print(tres.format())
if not tres.succeeded: print(tres.message)

In [ ]:
# Plot fit over data
fplot = FitPlot()
mplot.prepare(data, tmodel)
fplot.prepare(dplot, mplot)
fplot.plot()